In [1]:
import torch
from botorch.models import SingleTaskGP
from botorch.fit import fit_gpytorch_mll
from gpytorch.mlls import ExactMarginalLogLikelihood
from lume_model.variables import ScalarVariable, DistributionVariable
from lume_model.models.gp_model import GPModel
from gpytorch.kernels import RBFKernel
from gpytorch.kernels import ScaleKernel

# Multi-output example

In [2]:
torch.manual_seed(0)

# Create training data, 1 input, 3 outputs
train_x = torch.rand(5, 1)
train_y = torch.stack(
    (
        torch.sin(train_x * (2 * torch.pi)) + 0.1 * torch.randn(1),
        torch.cos(train_x * (2 * torch.pi)) + 0.1 * torch.randn(1),
        torch.sin(2 * train_x * (2 * torch.pi)) + 0.1 * torch.randn(1),
    ),
    dim=-1,
).squeeze(1)


# Initialize the GP model
rbf_kernel = ScaleKernel(RBFKernel())

model = SingleTaskGP(
    train_x.to(dtype=torch.double),
    train_y.to(dtype=torch.double),
    covar_module=rbf_kernel,
)

# Fit the model
mll = ExactMarginalLogLikelihood(model.likelihood, model)
fit_gpytorch_mll(mll)

# Derive posterior mean and variance
model.eval()
test_x = torch.linspace(0, 1, 10).reshape(-1, 1).to(dtype=torch.double)
posterior = model.posterior(test_x)

# Derive the posterior mean and variance for each output
mean = posterior.mean
variance = posterior.variance
print("Posterior Mean for each output:")
print(mean)
print("\nPosterior Variance for each output:")
print(variance)

Posterior Mean for each output:
tensor([[ 0.2397,  0.7271,  0.4191],
        [ 0.6916,  0.7995,  1.0390],
        [ 1.0555,  0.1417,  0.3111],
        [ 0.8914, -0.4485, -0.6847],
        [ 0.3121, -0.9120, -0.2559],
        [-0.1309, -0.7975,  0.1390],
        [-0.6172, -0.1708, -0.0208],
        [-0.9243,  0.1479, -0.1613],
        [-0.4698,  0.0957, -0.0618],
        [-0.0174,  0.0081,  0.0473]], dtype=torch.float64,
       grad_fn=<CloneBackward0>)

Posterior Variance for each output:
tensor([[0.1171, 0.1131, 0.1045],
        [0.0022, 0.0021, 0.0021],
        [0.0380, 0.0367, 0.0341],
        [0.0156, 0.0150, 0.0140],
        [0.0529, 0.0511, 0.0469],
        [0.0965, 0.0931, 0.0854],
        [0.2123, 0.2048, 0.1875],
        [0.0069, 0.0067, 0.0063],
        [0.3026, 0.2919, 0.2671],
        [0.4321, 0.4169, 0.3814]], dtype=torch.float64,
       grad_fn=<CloneBackward0>)


## LUME-Model import

In [4]:
# Define input variables
input_variables = [ScalarVariable(name="x")]

# Define output variables
output_variables = [
    DistributionVariable(name="output1"),
    DistributionVariable(name="output2"),
    DistributionVariable(name="output3"),
]

# Create lume_model instance
gp_lume_model = GPModel(
    model=model, input_variables=input_variables, output_variables=output_variables
)

### Evaluate model and run methods

In [5]:
input_dict = {"x": test_x.squeeze(1).to(dtype=torch.double)}

In [6]:
input_dict

{'x': tensor([0.0000, 0.1111, 0.2222, 0.3333, 0.4444, 0.5556, 0.6667, 0.7778, 0.8889,
         1.0000], dtype=torch.float64)}

In [7]:
# Evaluate function returns a dictionary mapping each output to a torch.distributions.Distribution
output_dict = gp_lume_model.evaluate(input_dict)

In [8]:
output_dict

{'output1': MultivariateNormal(loc: torch.Size([10]), covariance_matrix: torch.Size([10, 10])),
 'output2': MultivariateNormal(loc: torch.Size([10]), covariance_matrix: torch.Size([10, 10])),
 'output3': MultivariateNormal(loc: torch.Size([10]), covariance_matrix: torch.Size([10, 10]))}

In [9]:
test_prob = output_dict["output1"].sample(torch.Size([2]))

In [10]:
print("Posterior mean:", output_dict["output1"].mean)
print("Posterior Variance ", output_dict["output1"].variance)
print("Log Likelihood", output_dict["output1"].log_prob(test_prob))
print("Rsample ", output_dict["output1"].rsample(torch.Size([3])))

Posterior mean: tensor([ 0.2397,  0.6916,  1.0555,  0.8914,  0.3121, -0.1309, -0.6172, -0.9243,
        -0.4698, -0.0174], dtype=torch.float64, grad_fn=<ExpandBackward0>)
Posterior Variance  tensor([0.1171, 0.0022, 0.0380, 0.0156, 0.0529, 0.0965, 0.2123, 0.0069, 0.3026,
        0.4321], dtype=torch.float64, grad_fn=<ExpandBackward0>)
Log Likelihood tensor([2.8670, 1.3911], dtype=torch.float64, grad_fn=<SubBackward0>)
Rsample  tensor([[ 0.1371,  0.7029,  0.7924,  0.9986,  0.3675, -0.5925, -1.3643, -0.9383,
         -0.1128,  0.4696],
        [ 0.4477,  0.6424,  0.8958,  0.8927,  0.4303,  0.1575, -0.1585, -0.9204,
         -0.4836,  0.2818],
        [ 0.6743,  0.6468,  0.9740,  0.9098,  0.3655, -0.4459, -1.5865, -0.8818,
         -0.3347, -0.3025]], dtype=torch.float64, grad_fn=<ViewBackward0>)


### Outputs with original model

In [11]:
print("Posterior mean:", posterior.mean[:, 0])
print("Posterior Variance ", posterior.variance[:, 0])

Posterior mean: tensor([ 0.2397,  0.6916,  1.0555,  0.8914,  0.3121, -0.1309, -0.6172, -0.9243,
        -0.4698, -0.0174], dtype=torch.float64, grad_fn=<SelectBackward0>)
Posterior Variance  tensor([0.1171, 0.0022, 0.0380, 0.0156, 0.0529, 0.0965, 0.2123, 0.0069, 0.3026,
        0.4321], dtype=torch.float64, grad_fn=<SelectBackward0>)


# A 3D Rosenbrock example for GPModel class

## Create and train a GP

In [12]:
# Define the 3D Rosenbrock function
def rosenbrock(X):
    x1, x2, x3 = X[..., 0], X[..., 1], X[..., 2]
    return (
        (1 - x1) ** 2
        + 100 * (x2 - x1**2) ** 2
        + (1 - x2) ** 2
        + 100 * (x3 - x2**2) ** 2
    )

In [13]:
# Generate training data
train_x = torch.rand(20, 3) * 4 - 2  # 20 points in 3D space, scaled to [-2, 2]
train_y = rosenbrock(train_x).unsqueeze(-1)  # Compute the Rosenbrock function values

# Define the GP model
gp_model = SingleTaskGP(train_x.to(dtype=torch.double), train_y.to(dtype=torch.double))

# Fit the model
mll = ExactMarginalLogLikelihood(gp_model.likelihood, gp_model)
fit_gpytorch_mll(mll)

# Evaluate the model on test data
test_x = torch.rand(10, 3) * 4 - 2  # 10 new points in 3D space
gp_model.eval()
posterior = gp_model.posterior(test_x)

# Get the mean and variance of the posterior
mean = posterior.mean
variance = posterior.variance

print("Posterior mean: ", mean)
print("Posterior variance: ", variance)

Posterior mean:  tensor([[ 606.3839],
        [ 685.3384],
        [1196.5879],
        [1049.7149],
        [ 258.3421],
        [1224.6845],
        [ 926.0569],
        [ 801.6735],
        [ 334.0830],
        [ 529.1804]], dtype=torch.float64, grad_fn=<UnsqueezeBackward0>)
Posterior variance:  tensor([[ 28350.0389],
        [246126.2279],
        [129956.1835],
        [ 73671.8765],
        [ 27067.5129],
        [154205.9639],
        [180241.9565],
        [206996.4155],
        [ 44488.5639],
        [ 73535.2854]], dtype=torch.float64, grad_fn=<UnsqueezeBackward0>)


/Users/smiskov/miniconda3/envs/lume-latest/lib/python3.10/site-packages/botorch/models/utils/assorted.py:264: InputDataWarning: Data (input features) is not contained to the unit cube. Please consider min-max scaling the input data.
  check_min_max_scaling(


## LUME-Model import

In [15]:
# Define input variables
input_variables = [
    ScalarVariable(name="x1"),
    ScalarVariable(name="x2"),
    ScalarVariable(name="x3"),
]

# Define output variables
output_variables = [DistributionVariable(name="output1")]

# Create lume_model instance
gp_lume_model = GPModel(
    model=gp_model, input_variables=input_variables, output_variables=output_variables
)

#### Evaluate model and run methods

In [16]:
input_x = torch.rand(3, 3) * 4 - 2  # 3 new points in 3D space
input_dict = {
    "x1": input_x[:, 0].to(dtype=torch.double),
    "x2": input_x[:, 1].to(dtype=torch.double),
    "x3": input_x[:, 2].to(dtype=torch.double),
}

In [17]:
# Evaluate function returns a dictionary mapping each output to a torch.distributions.Distribution
lume_dist = gp_lume_model.evaluate(input_dict)["output1"]

In [18]:
rand_test = torch.rand(1, 3)

In [19]:
print("Posterior mean:", lume_dist.mean)
print("Posterior Variance ", lume_dist.variance)
print("Log Likelihood", lume_dist.log_prob(rand_test))
print("Rsample ", lume_dist.rsample(torch.Size([3])))

Posterior mean: tensor([ 144.1534,  750.6190, 1551.5102], dtype=torch.float64,
       grad_fn=<ExpandBackward0>)
Posterior Variance  tensor([107860.9063, 105648.6779,  77374.0159], dtype=torch.float64,
       grad_fn=<ExpandBackward0>)
Log Likelihood tensor([-35.7136], dtype=torch.float64, grad_fn=<SubBackward0>)
Rsample  tensor([[ 225.9312,  858.3771, 2018.9276],
        [ 315.6748, 1014.5847, 1271.6913],
        [ 368.0957, 1014.4358, 2090.0455]], dtype=torch.float64,
       grad_fn=<ViewBackward0>)


### Outputs with original model

In [20]:
posterior = gp_model.posterior(input_x)
botorch_dist = posterior.distribution

In [21]:
print("Posterior mean:", botorch_dist.mean)
print("Posterior Variance ", botorch_dist.variance)
print("Log Likelihood", botorch_dist.log_prob(rand_test))
print("Rsample ", botorch_dist.rsample(torch.Size([3])))

Posterior mean: tensor([ 144.1534,  750.6190, 1551.5102], dtype=torch.float64,
       grad_fn=<AddBackward0>)
Posterior Variance  tensor([107860.9063, 105648.6779,  77374.0159], dtype=torch.float64,
       grad_fn=<ExpandBackward0>)
Log Likelihood tensor([-35.7136], dtype=torch.float64, grad_fn=<SubBackward0>)
Rsample  tensor([[  76.0090,  322.3840, 1341.8768],
        [ 129.2442,  821.3236, 1278.1483],
        [-149.8472,  711.9353, 1303.4597]], dtype=torch.float64,
       grad_fn=<ViewBackward0>)
